In [ ]:
import rdkit
from rdkit import Chem
import pandas as pd
from rdkit.Chem import RDConfig, rdMolDescriptors, Descriptors, PandasTools
from syba.syba import SybaClassifier
import os
import sys
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
import sascorer

root = os.path.dirname(os.getcwd())

## Compute other scores and metrics

In [ ]:
def calc_SAscore(mol):
    return sascorer.calculateScore(mol)

scscore_path = os.path.join(root, 'scscore')
sys.path.append(scscore_path)
from scscore.standalone_model_numpy import SCScorer
model_sc = SCScorer()
model_sc.restore(os.path.join(scscore_path, 'models', 'full_reaxys_model_2048bool', 'model.ckpt-10654.as_numpy.pickle'), FP_len=2048)

def calc_SCscore(smi):
    smi, score = model_sc.get_score_from_smi(smi)
    return score

syba = SybaClassifier()
syba.fitDefaultScore()

def calc_SYBA(smi, mol = None):
    # takes very long
    score = syba.predict(smi=smi, mol=mol)
    return score

def calc_logP(mol):
    return rdMolDescriptors.CalcCrippenDescriptors(mol)[0]

def calc_QED(mol):
    return Descriptors.qed(mol)

def calc_MW(mol):
    return Descriptors.MolWt(mol)

def calc_numheavyatoms(mol):
    return mol.GetNumHeavyAtoms()

In [ ]:
df_graph = pd.read_csv('/home/rebecca/code/intuitive-sc/results/uspto_cjhif_combo_test_reorder_combo_train_graph_sub100000_ep20_2023-07-11_scores.csv')
df_fp = pd.read_csv('/home/rebecca/code/intuitive-sc/results/uspto_cjhif_combo_test_reorder_combo_train_fp_sub100000_ep20_2023-07-07_scores.csv')
df_graph.head()

In [ ]:
# add mols
PandasTools.AddMoleculeColumnToFrame(df_graph, smilesCol='smiles_i', molCol='mol_i')
PandasTools.AddMoleculeColumnToFrame(df_fp, smilesCol='smiles_i', molCol='mol_i')

In [ ]:
from pathos import multiprocessing as mp
from tqdm import tqdm
import numpy as np

def chunked_parallel(input_list, function, chunks = 1000, max_cpu = 20): 
    """chunked_parallel

    Args:
        input_list : list of objects to apply function
        function : Callable with 1 input and returning a single value
        chunks: number of chunks
        max_cpu: Max num cpus
    """

    cpus = min(mp.cpu_count(), max_cpu)
    pool = mp.Pool(processes = cpus)

    def batch_func(list_inputs): 
        outputs = []
        for i in list_inputs: 
            outputs.append(function(i))
        return outputs

    list_len = len(input_list)
    num_chunks = min(list_len, chunks)
    step_size = len(input_list) // num_chunks

    chunked_list = [input_list[i:i+step_size] 
                    for i in range(0, len(input_list), step_size)]

    list_outputs = list(tqdm(pool.imap(batch_func, chunked_list),
                             total=num_chunks))
    full_output = [j for i in list_outputs for j in i]
    return full_output

In [ ]:
def calc_all_scores(df, mol_col = 'mol_i', smiles_col = 'smiles_i'):
    df['SA_score'] = chunked_parallel(df[mol_col], calc_SAscore)
    df['SC_score'] = chunked_parallel(df[smiles_col], calc_SCscore)
    df['SYBA_score'] = chunked_parallel(df[smiles_col], calc_SYBA)
    df['logP'] = chunked_parallel(df[mol_col], calc_logP)
    df['QED'] = chunked_parallel(df[mol_col], calc_QED)
    df['MW'] = chunked_parallel(df[mol_col], calc_MW)
    df['numheavyatoms'] = chunked_parallel(df[mol_col], calc_numheavyatoms)
    return df

In [ ]:
df_fp = calc_all_scores(df_fp)
df_fp.head()

In [ ]:
df_graph = calc_all_scores(df_graph)
df_graph.head()

In [ ]:
df_ra_fp = pd.read_csv('/home/rebecca/code/intuitive-sc/results/combo_test_fp_sub100000_ep20_2023-07-07_RAscores.csv')
df_ra_graph = pd.read_csv('/home/rebecca/code/intuitive-sc/results/combo_test_graph_sub100000_ep20_2023-07-11_RAscores.csv')
df_ra_fp.head()

In [ ]:
df_fp = pd.read_csv('/home/rebecca/code/intuitive-sc/results/combo_test_fp_sub100000_ep20_2023-07-07_scores_all.csv')
df_graph = pd.read_csv('/home/rebecca/code/intuitive-sc/results/combo_test_graph_sub100000_ep20_2023-07-11_scores_all.csv')
df_fp.head()

In [ ]:
PandasTools.AddMoleculeColumnToFrame(df_fp, smilesCol='smiles_i', molCol='mol_i')
PandasTools.AddMoleculeColumnToFrame(df_graph, smilesCol='smiles_i', molCol='mol_i')

In [ ]:
# add RA score to full df
df_fp['RAscore'] = df_ra_fp['RAscore']
df_graph['RAscore'] = df_ra_graph['RAscore']

In [ ]:
df_fp.drop(columns=['mol_i'], inplace=True)
df_fp.to_csv('/home/rebecca/code/intuitive-sc/results/combo_test_fp_sub100000_ep20_2023-07-07_scores_all.csv', index=False)
df_graph.drop(columns=['mol_i'], inplace=True)
df_graph.to_csv('/home/rebecca/code/intuitive-sc/results/combo_test_graph_sub100000_ep20_2023-07-11_scores_all.csv', index=False)

## Analysis

In [ ]:
df_fp = pd.read_csv('/home/rebecca/code/intuitive-sc/results/syba_testset_fpscored.csv')
df_graph = pd.read_csv('/home/rebecca/code/intuitive-sc/results/syba_testset_graphscored.csv')
smi_col = 'smi'

In [ ]:
# deduplicate for analysis
df_fp = df_fp.drop_duplicates(subset=[smi_col])
df_graph = df_graph.drop_duplicates(subset=[smi_col])

### Plot correlations

In [ ]:
from scipy.stats import pearsonr

def plot_scatter(ax, x1, y1, x2, y2, ylabel):
    pcc_fp = pearsonr(x1, y1)[0]
    pcc_graph = pearsonr(x2, y2)[0]
    ax.scatter(x1, y1, alpha = 0.3, label = f'FP (PCC={pcc_fp:.3f})', s=2);
    ax.scatter(x2, y2, alpha = 0.3, label = f'Graph (PCC={pcc_graph:.3f})', s=2);
    ax.set_xlabel('ours');
    ax.set_ylabel(ylabel);
    ax.legend(loc='lower left');
    ax.set_title(ylabel);
    return pcc_fp, pcc_graph

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2,4,figsize=(17, 10), sharex=True)

pcc_fp_new_sa, pcc_graph_new_sa = plot_scatter(axs[0,0], df_fp['score'], df_fp['SA_score'], df_graph['score'], df_graph['SA_score'], 'SA score')
pcc_fp_new_sc, pcc_graph_new_sc = plot_scatter(axs[0,1], df_fp['score'], df_fp['SC_score'], df_graph['score'], df_graph['SC_score'], 'SC score')
pcc_fp_new_syba, pcc_graph_new_syba = plot_scatter(axs[0,2], df_fp['score'], df_fp['SYBA_score'], df_graph['score'], df_graph['SYBA_score'], 'SYBA score')
pcc_fp_new_ra, pcc_graph_new_ra = plot_scatter(axs[0,3], df_fp['score'], df_fp['RAscore'], df_graph['score'], df_graph['RAscore'], 'RA score')
pcc_fp_new_logp, pcc_graph_new_logp = plot_scatter(axs[1,0], df_fp['score'], df_fp['logP'], df_graph['score'], df_graph['logP'], 'logP')
pcc_fp_new_qed, pcc_graph_new_qed = plot_scatter(axs[1,1], df_fp['score'], df_fp['QED'], df_graph['score'], df_graph['QED'], 'QED')
pcc_fp_new_mw, pcc_graph_new_mw = plot_scatter(axs[1,2], df_fp['score'], df_fp['MW'], df_graph['score'], df_graph['MW'], 'MW')
pcc_fp_new_natoms, pcc_graph_new_natoms = plot_scatter(axs[1,3], df_fp['score'], df_fp['numheavyatoms'], df_graph['score'], df_graph['numheavyatoms'], 'num heavy atoms')

In [ ]:
# correlations between old scores themselves

fig, axs = plt.subplots(2,3,figsize=(15, 10))

pccsa_sc = pearsonr(df_fp['SA_score'], df_fp['SC_score'])[0]
pccsa_syba = pearsonr(df_fp['SA_score'], df_fp['SYBA_score'])[0]
pccsa_ra = pearsonr(df_fp['SA_score'], df_fp['RAscore'])[0]
axs[0,0].scatter(df_fp['SA_score'], df_fp['SC_score'], alpha = 0.5, label = f'PCC={pccsa_sc:.3f}', s=2, color='grey');
axs[0,0].set_xlabel('SA score');
axs[0,0].set_ylabel('SC score');
axs[0,0].text(0.7, 0.05, f'PCC={pccsa_sc:.3f}', transform=axs[0,0].transAxes)

axs[0,1].scatter(df_fp['SA_score'], df_fp['SYBA_score'], alpha = 0.5, label = f'PCC={pccsa_syba:.3f}', s=2, color='grey');
axs[0,1].set_xlabel('SA score');
axs[0,1].set_ylabel('SYBA score');
axs[0,1].text(0.7, 0.05, f'PCC={pccsa_syba:.3f}', transform=axs[0,1].transAxes)

axs[0,2].scatter(df_fp['SA_score'], df_fp['RAscore'], alpha = 0.5, label = f'PCC={pccsa_ra:.3f}', s=2, color='grey');
axs[0,2].set_xlabel('SA score');
axs[0,2].set_ylabel('RA score');
axs[0,2].text(0.7, 0.05, f'PCC={pccsa_ra:.3f}', transform=axs[0,2].transAxes)

pccsc_syba = pearsonr(df_fp['SC_score'], df_fp['SYBA_score'])[0]
pccsc_ra = pearsonr(df_fp['SC_score'], df_fp['RAscore'])[0]
axs[1,0].scatter(df_fp['SC_score'], df_fp['SYBA_score'], alpha = 0.5, label = f'PCC={pccsc_syba:.3f}', s=2, color='grey');
axs[1,0].set_xlabel('SC score');
axs[1,0].set_ylabel('SYBA score');
axs[1,0].text(0.7, 0.05, f'PCC={pccsc_syba:.3f}', transform=axs[1,0].transAxes)

axs[1,1].scatter(df_fp['SC_score'], df_fp['RAscore'], alpha = 0.5, label = f'PCC={pccsc_ra:.3f}', s=2, color='grey');
axs[1,1].set_xlabel('SC score');
axs[1,1].set_ylabel('RA score');
axs[1,1].text(0.7, 0.05, f'PCC={pccsc_ra:.3f}', transform=axs[1,1].transAxes)

pccsyba_ra = pearsonr(df_fp['SYBA_score'], df_fp['RAscore'])[0]
axs[1,2].scatter(df_fp['SYBA_score'], df_fp['RAscore'], alpha = 0.5, label = f'PCC={pccsyba_ra:.3f}', s=2, color='grey');
axs[1,2].set_xlabel('SYBA score');
axs[1,2].set_ylabel('RA score');
axs[1,2].text(0.7, 0.05, f'PCC={pccsyba_ra:.3f}', transform=axs[1,2].transAxes)

In [ ]:
# barplots of correlations
import matplotlib.patches as mpatches
pccs_fp = [pcc_fp_new_sa, pcc_fp_new_sc, pcc_fp_new_syba, pcc_fp_new_ra, pcc_fp_new_logp, pcc_fp_new_qed, pcc_fp_new_mw, pcc_fp_new_natoms]
pccs_graph = [pcc_graph_new_sa, pcc_graph_new_sc, pcc_graph_new_syba, pcc_graph_new_ra, pcc_graph_new_logp, pcc_graph_new_qed, pcc_graph_new_mw, pcc_graph_new_natoms]
pccs_old = [pccsa_sc, pccsa_syba, pccsa_ra, pccsc_syba, pccsc_ra, pccsyba_ra]

ppcs_all = pccs_fp + pccs_graph + pccs_old

# color list based on fp, graph, old
colors = ['tab:blue']*8 + ['tab:orange']*8 + ['grey']*6

# legends
legends = ['FP ~ SA', 'FP ~ SC', 'FP ~ SYBA', 'FP ~ RA', 'FP ~ logP', 'FP ~ QED', 'FP ~ MW', 'FP ~ #atoms', 'Graph ~ SA', 'Graph ~ SC', 'Graph ~ SYBA', 'Graph ~ RA', 'Graph ~ logP', 'Graph ~ QED', 'Graph ~ MW', 'Graph ~ #atoms', 'SA ~ SC', 'SA ~ SYBA', 'SA ~ RA', 'SC ~ SYBA', 'SC ~ RA', 'SYBA ~ RA']

# get absolute values
ppcs_all = [abs(x) for x in ppcs_all]

# make dictionary with colors
color_dict = dict(zip(ppcs_all, colors))
legends_dict = dict(zip(ppcs_all, legends))

# sort by descending order
ppcs_all.sort(reverse=True)

# get colors and legends in same order
colors_sorted = [color_dict[x] for x in ppcs_all]
legends_sorted = [legends_dict[x] for x in ppcs_all]

# plot absolute values
plt.figure(figsize=(10,5));
plt.bar(range(len(ppcs_all)), ppcs_all, color=colors_sorted);
plt.xticks(range(len(ppcs_all)), legends_sorted, rotation=45, ha='right');
plt.ylabel('abs PCC');
plt.title('PCCs for all scores');

graph_patch = mpatches.Patch(color='tab:orange', label='Graph');
fp_patch = mpatches.Patch(color='tab:blue', label='FP');
old_patch = mpatches.Patch(color='grey', label='Known scores');
plt.legend(handles=[graph_patch, fp_patch, old_patch]);


### Visualize examples

In [ ]:
# display top and bottom 10 molecules with their scores
df_fp.sort_values(by=['score'], inplace=True)
df_graph.sort_values(by=['score'], inplace=True)

In [ ]:
n = 10
bottom_smi = df_fp[smi_col].head(n).tolist()
bottom_mols = [Chem.MolFromSmiles(smi) for smi in bottom_smi]
bottom_scores = df_fp['score'].head(n).tolist()
bottom_sa = df_fp['SA_score'].head(n).tolist()
bottom_sc = df_fp['SC_score'].head(n).tolist()
bottom_syba = df_fp['SYBA_score'].head(n).tolist()
bottom_ra = df_fp['RAscore'].head(n).tolist()
# create legend texts for every mol by zipping the scores together
legends = [f'new: {new:.2f}\n SA: {sa:.2f}\t SC: {sc:.2f}\n SYBA: {syba:.2f}\t RA: {ra:.2f}'
              for new, sa, sc, syba, ra in zip(bottom_scores, bottom_sa, bottom_sc, bottom_syba, bottom_ra)]
Chem.Draw.MolsToGridImage(bottom_mols, molsPerRow=5, legends=legends, subImgSize=(300,400))

In [ ]:
top_smi = df_fp[smi_col].tail(10).tolist()
top_mols = [Chem.MolFromSmiles(smi) for smi in top_smi]
top_scores = df_fp['score'].tail(10).tolist()
top_sa = df_fp['SA_score'].tail(10).tolist()
top_sc = df_fp['SC_score'].tail(10).tolist()
top_syba = df_fp['SYBA_score'].tail(10).tolist()
top_ra = df_fp['RAscore'].tail(10).tolist()
# create legend texts for every mol by zipping the scores together
legends = [f'new: {new:.2f}\n SA: {sa:.2f}\t SC: {sc:.2f}\n SYBA: {syba:.2f}\t RA: {ra:.2f}'
              for new, sa, sc, syba, ra in zip(top_scores, top_sa, top_sc, top_syba, top_ra)]
Chem.Draw.MolsToGridImage(top_mols, molsPerRow=5, legends=legends, subImgSize=(300,400))

In [ ]:
bottom_smi = df_graph[smi_col].head(n).tolist()
bottom_mols = [Chem.MolFromSmiles(smi) for smi in bottom_smi]
bottom_scores = df_graph['score'].head(n).tolist()
bottom_sa = df_graph['SA_score'].head(n).tolist()
bottom_sc = df_graph['SC_score'].head(n).tolist()
bottom_syba = df_graph['SYBA_score'].head(n).tolist()
bottom_ra = df_graph['RAscore'].head(n).tolist()
# create legend texts for every mol by zipping the scores together
legends = [f'new: {new:.2f}\n SA: {sa:.2f}\t SC: {sc:.2f}\n SYBA: {syba:.2f}\t RA: {ra:.2f}'
              for new, sa, sc, syba, ra in zip(bottom_scores, bottom_sa, bottom_sc, bottom_syba, bottom_ra)]
Chem.Draw.MolsToGridImage(bottom_mols, molsPerRow=5, legends=legends, subImgSize=(300,400))

In [ ]:
top_smi = df_graph[smi_col].tail(10).tolist()
top_mols = [Chem.MolFromSmiles(smi) for smi in top_smi]
top_scores = df_graph['score'].tail(10).tolist()
top_sa = df_graph['SA_score'].tail(10).tolist()
top_sc = df_graph['SC_score'].tail(10).tolist()
top_syba = df_graph['SYBA_score'].tail(10).tolist()
top_ra = df_graph['RAscore'].tail(10).tolist()
# create legend texts for every mol by zipping the scores together
legends = [f'new: {new:.2f}\n SA: {sa:.2f}\t SC: {sc:.2f}\n SYBA: {syba:.2f}\t RA: {ra:.2f}'
              for new, sa, sc, syba, ra in zip(top_scores, top_sa, top_sc, top_syba, top_ra)]
Chem.Draw.MolsToGridImage(top_mols, molsPerRow=5, legends=legends, subImgSize=(300,400))

In [ ]:
# visualize those where the difference is the largest between FP and graph
df_combo = pd.merge(df_fp, df_graph, on=smi_col, suffixes=('_fp', '_graph'))
df_combo = df_combo[[smi_col, 'score_fp', 'score_graph']]
# drop columns that are not needed
df_combo['diff'] = df_combo['score_fp'] - df_combo['score_graph']
# absolute value
df_combo['abs_diff'] = df_combo['diff'].abs()
df_combo.head()

In [ ]:
# sort by descending order
df_combo.sort_values(by=['abs_diff'], inplace=True, ascending=False)
df_combo.head()

In [ ]:
# show top 10
top_smi = df_combo[smi_col].head(10).tolist()
top_mols = [Chem.MolFromSmiles(smi) for smi in top_smi]
top_scores_fp = df_combo['score_fp'].head(10).tolist()
top_scores_graph = df_combo['score_graph'].head(10).tolist()
top_diff = df_combo['diff'].head(10).tolist()
# create legend texts for every mol by zipping the scores together
legends = [f'FP: {fp:.2f}\n Graph: {graph:.2f}\n Diff: {diff:.2f}'
                for fp, graph, diff in zip(top_scores_fp, top_scores_graph, top_diff)]
Chem.Draw.MolsToGridImage(top_mols, molsPerRow=5, legends=legends, subImgSize=(300,400))

In [ ]:
# show bottom 10
bottom_smi = df_combo[smi_col].tail(10).tolist()
bottom_mols = [Chem.MolFromSmiles(smi) for smi in bottom_smi]
bottom_scores_fp = df_combo['score_fp'].tail(10).tolist()
bottom_scores_graph = df_combo['score_graph'].tail(10).tolist()
bottom_diff = df_combo['diff'].tail(10).tolist()
# create legend texts for every mol by zipping the scores together
legends = [f'FP: {fp:.2f}\n Graph: {graph:.2f}\n Diff: {diff:.2f}'
                for fp, graph, diff in zip(bottom_scores_fp, bottom_scores_graph, bottom_diff)]
Chem.Draw.MolsToGridImage(bottom_mols, molsPerRow=5, legends=legends, subImgSize=(300,400))

### Distribution of values

In [ ]:
# plot histograms
fig, axs = plt.subplots(2,3,figsize=(15, 10))

bins = 50
alpha = 0.5
density=False
axs[0,0].hist(df_fp['score'], bins=bins, alpha=alpha, label='FP',density=density)
axs[0,0].set_xlabel('new');
axs[0,0].hist(df_graph['score'], bins=bins, alpha=alpha, label='graph', density=density)
axs[0,0].legend(loc='upper left');
axs[0,1].hist(df_fp['SA_score'], bins=bins, alpha=alpha, density=density, color='k')
axs[0,1].set_xlabel('SA score');
axs[0,2].hist(df_fp['SC_score'], bins=bins, alpha=alpha, density=density, color='k')
axs[0,2].set_xlabel('SC score');
axs[1,0].hist(df_fp['SYBA_score'], bins=bins, alpha=alpha, density=density, color='k')
axs[1,0].set_xlabel('SYBA score');
axs[1,1].hist(df_fp['RAscore'], bins=bins, alpha=alpha, density=density, color='k')
axs[1,1].set_xlabel('RA score');

### Classification power on hard/easy to synthesize compounds
Based on test sets from SYBA classified into ES (easy to synthesis) and HS (hard to synthesis) compounds.
Two sources are included: MC (labelled by medicinal chemists) and CP (classified by complexity scores).

In [ ]:
# HS gets label 0, ES gets label 1 based on column label
df_fp['label_bin'] = df_fp['label'].apply(lambda x: 0 if x == 'HS' else 1)
df_graph['label_bin'] = df_graph['label'].apply(lambda x: 0 if x == 'HS' else 1)

In [ ]:
# calculate ROC AUC, acc, SN and SP
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.metrics import confusion_matrix

roc_curve_fp = roc_curve(df_fp['label_bin'], df_fp['score'])
roc_curve_graph = roc_curve(df_graph['label_bin'], df_graph['score'])
roc_curve_sa = roc_curve(df_fp['label_bin'], df_fp['SA_score'], pos_label=0)
roc_curve_sc = roc_curve(df_fp['label_bin'], df_fp['SC_score'], pos_label=0)
roc_curve_syba = roc_curve(df_fp['label_bin'], df_fp['SYBA_score'])
roc_curve_ra = roc_curve(df_fp['label_bin'], df_fp['RAscore'])
auc_fp = auc(roc_curve_fp[0], roc_curve_fp[1])
auc_graph = auc(roc_curve_graph[0], roc_curve_graph[1])
auc_sa = auc(roc_curve_sa[0], roc_curve_sa[1])
auc_sc = auc(roc_curve_sc[0], roc_curve_sc[1])
auc_syba = auc(roc_curve_syba[0], roc_curve_syba[1])
auc_ra = auc(roc_curve_ra[0], roc_curve_ra[1])


In [ ]:
# plot ROC curve
fig, ax = plt.subplots(figsize=(7, 5))
ax.plot(roc_curve_fp[0], roc_curve_fp[1], label=f'FP (AUC = {auc_fp:.2f})')
ax.plot(roc_curve_graph[0], roc_curve_graph[1], label= f'graph (AUC = {auc_graph:.2f})')
ax.plot(roc_curve_sa[0], roc_curve_sa[1], label=f'SA (AUC = {auc_sa:.2f})')
ax.plot(roc_curve_sc[0], roc_curve_sc[1], label=f'SC (AUC = {auc_sc:.2f})')
ax.plot(roc_curve_syba[0], roc_curve_syba[1], label=f'SYBA (AUC = {auc_syba:.2f})')
ax.plot(roc_curve_ra[0], roc_curve_ra[1], label=f'RA (AUC = {auc_ra:.2f})')
ax.plot([0, 1], [0, 1], color='navy', linestyle='--')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC curve')
ax.legend(loc="lower right");


In [ ]:
# calculate accuracy, SN and SP
# get splitting score by balancing SN and SP (youden index = max(SN + SP - 1))
def get_splitting_score(y_true, y_pred, pos_label=1):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred, pos_label=pos_label)
    # calculate youden index
    youden = tpr - fpr
    # locate the index
    idx = np.argmax(youden)
    return thresholds[idx]

def get_acc_sn_sp(y_true, y_pred, threshold, bigger = True):
    if bigger:
        y_pred_bin = np.where(y_pred > threshold, 1, 0)
    else:
        y_pred_bin = np.where(y_pred < threshold, 1, 0)
    acc = accuracy_score(y_true, y_pred_bin)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred_bin).ravel()
    sn = tp / (tp + fn)
    sp = tn / (tn + fp)
    return acc, sn, sp

threshold_fp = get_splitting_score(df_fp['label_bin'], df_fp['score'])
threshold_graph = get_splitting_score(df_graph['label_bin'], df_graph['score'])
threshold_sa = get_splitting_score(df_fp['label_bin'], df_fp['SA_score'], pos_label=0)
threshold_sc = get_splitting_score(df_fp['label_bin'], df_fp['SC_score'], pos_label=0)   
threshold_syba = get_splitting_score(df_fp['label_bin'], df_fp['SYBA_score'])
threshold_ra = get_splitting_score(df_fp['label_bin'], df_fp['RAscore'])

acc_fp, sn_fp, sp_fp = get_acc_sn_sp(df_fp['label_bin'], df_fp['score'], threshold_fp)
acc_graph, sn_graph, sp_graph = get_acc_sn_sp(df_graph['label_bin'], df_graph['score'], threshold_graph)
acc_sa, sn_sa, sp_sa = get_acc_sn_sp(df_fp['label_bin'], df_fp['SA_score'], threshold_sa, bigger=False)
acc_sc, sn_sc, sp_sc = get_acc_sn_sp(df_fp['label_bin'], df_fp['SC_score'], threshold_sc, bigger=False)
acc_syba, sn_syba, sp_syba = get_acc_sn_sp(df_fp['label_bin'], df_fp['SYBA_score'], threshold_syba)
acc_ra, sn_ra, sp_ra = get_acc_sn_sp(df_fp['label_bin'], df_fp['RAscore'], threshold_ra)

# plot accuracy, SN and SP
fig, ax = plt.subplots(figsize=(7, 5))
ax.bar([0, 1, 2, 3, 4, 5], [acc_fp, acc_graph, acc_sa, acc_sc, acc_syba, acc_ra], color='k')
ax.set_xticks([0, 1, 2, 3, 4, 5])
ax.set_xticklabels(['FP', 'graph', 'SA', 'SC', 'SYBA', 'RA'])
ax.set_ylabel('Accuracy')
ax.set_title('Accuracy, SN and SP')
ax.set_ylim(0, 1)
ax.set_yticks(np.arange(0, 1.1, 0.1))
ax.set_yticklabels(['0', '10%', '20%', '30%', '40%', '50%', '60%', '70%', '80%', '90%', '100%'])
ax2 = ax.twinx()
ax2.plot([0, 1, 2, 3, 4, 5], [sn_fp, sn_graph, sn_sa, sn_sc, sn_syba, sn_ra], color='r', marker='o', label='SN')
ax2.plot([0, 1, 2, 3, 4, 5], [sp_fp, sp_graph, sp_sa, sp_sc, sp_syba, sp_ra], color='b', marker='o', label='SP')
ax2.set_ylim(0, 1)
ax2.set_yticks(np.arange(0, 1.1, 0.1))
ax2.set_yticklabels(['0', '10%', '20%', '30%', '40%', '50%', '60%', '70%', '80%', '90%', '100%'])
ax2.set_ylabel('SN and SP')
ax2.legend(loc='upper left');
